In [29]:
import json
from pathlib import Path

import pandas as pd

In [30]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-semi/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

29 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.5,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0},
  'run': 1},
 'metrics': {'exact_match': 0.34,
  'f1': 0.43138207471540807,
  'fuzzy_match': 0.39666666666666667,
  '2hops': {'exact_match': 0.34,
   'f1': 0.41819047619047617,
   'fuzzy_match': 0.38},
  '3hops': {'exact_match': 0.27,
   'f1': 0.3541065416065416,
   'fuzzy_match': 0.33},
  '4hops': {'exact_match': 0.41,
   'f1': 0.5218492063492063,
   'fuzzy_match': 0.48}}}

In [31]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")  &
    (df["params.qa.few_shot_examples"].str.contains("semi")) 
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

24 experiments after preprocessing


In [32]:
def parse_qa_technique(system_prompt_filename):
    return system_prompt_filename.split("/")[0]

df["params.qa.technique"] = df["params.qa.system_prompt"].map(parse_qa_technique)

In [33]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,0.5,"(1,)"
1,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,0.5,"(1,)"
2,ccot/format-thought.txt,cq.txt,semi/ccot.json,2,0.5,"(1,)"
3,ccot/format-thought.txt,cq.txt,semi/ccot.json,3,0.5,"(1,)"
4,ccot/original.txt,cq.txt,semi/ccot-original.json,0,0.5,"(1,)"
5,ccot/original.txt,cq.txt,semi/ccot-original.json,1,0.5,"(1,)"
6,ccot/original.txt,cq.txt,semi/ccot-original.json,2,0.5,"(1,)"
7,ccot/original.txt,cq.txt,semi/ccot-original.json,3,0.5,"(1,)"
8,cot/format-thought.txt,cq.txt,semi/cot.json,0,0.5,"(1,)"
9,cot/format-thought.txt,cq.txt,semi/cot.json,1,0.5,"(1,)"


In [34]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	ccot/original.txt
	cot/format-thought.txt
	cot/original.txt
	cte/format-triplets-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/ccot-original.json
	semi/cot.json
	semi/cot-original.json
	semi/cte.json
	semi/direct.json
- params.qa.n_shot
	0
	1
	2
	3
- params.qa.temperature
	0.5
- params.run
	(1,)


In [35]:
df.to_json('exps.jsonl', orient='records', lines=True)